In [1]:
import os

train_done = True
study_id = 'original'
data_id = 'pan'

size = '100p'
per = 'all'
data_ = 'TransMut'
encoding_ = 'embedding'


id_ = 'OF_TData_pad14_{}_EmbeddingAttention'.format(size)

data_dir = '/home/s202357/thesis/transmut/data/transmut_github/'
model_folder = '/home/s202357/thesis/transmut/pipeline/{}/models/'.format(study_id)

result_folder = '/home/s202357/thesis/transmut/pipeline/{}/results/'.format(study_id)

plot_name = result_folder + id_+ '_plot_'
df_name = result_folder + id_+ '_df_'

if not os.path.exists(model_folder):
        print('creating dirs')
        os.makedirs(model_folder)
        os.makedirs(result_folder)
else:
        print('dirs exist')


dirs exist


In [ ]:
!gpustat

In [53]:
os.environ['CUDA_VISIBLE_DEVICES']='4'

In [54]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, accuracy_score
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
#from sklearn.metrics import classification_report
#from sklearn.utils import class_weight

import gc
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

#import framework as fr

In [55]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    use_cuda = True
    print('There are %d GPU(s) avalable.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) avalable.


In [56]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [57]:
def best_treshold(df_results):
    acc_best = -999
    best_i = -999

    for i in np.arange(0, 1, 0.1):
        df_results['pred_binary'] = np.where((df_results.pred > i) , 1, 0)
        acc = accuracy_score(df_results['target'], df_results['pred_binary'])
        if acc > acc_best:
            df = df_results
            best_i = i
            acc_best = acc
            
    return acc_best, best_i, df


def best_treshold(df_results, t= []):
    acc_best = -999
    best_i = -999

    if t == []:
        for i in np.arange(0, 1, 0.1):
            df_results['pred_binary'] = np.where((df_results.pred > i) , 1, 0)
            acc = accuracy_score(df_results['target'], df_results['pred_binary'])
            if acc > acc_best:
                df = df_results
                best_i = i
                acc_best = acc
    else:
        best_i = t[0]
        df_results['pred_binary'] = np.where((df_results.pred > t[0]) , 1, 0)
        df = df_results
    return acc_best, best_i, df

def pkl(id_, n_layers, n_heads, model_num, d_model):
    return '{}_d{}_layer{}_multihead{}_MODEL{}.pkl'.format(id_, d_model, n_layers, n_heads, model_num)


def make_data(data):
    pep_inputs, hla_inputs, labels = [], [], []
    for pep, hla, label in zip(data.peptide, data.HLA_sequence, data.label):
        pep, hla = pep.ljust(pep_max_len, '-'), hla.ljust(hla_max_len, '-')
        pep_input = [[vocab[n] for n in pep]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
        hla_input = [[vocab[n] for n in hla]]
        pep_inputs.extend(pep_input)
        hla_inputs.extend(hla_input)
        labels.append(label)
    return torch.LongTensor(pep_inputs), torch.LongTensor(hla_inputs), torch.LongTensor(labels)

class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs, labels):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs
        self.labels = labels

    def __len__(self):
        return self.pep_inputs.shape[0]

    def __getitem__(self, idx):
        return self.pep_inputs[idx], self.hla_inputs[idx], self.labels[idx]

In [70]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]


class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn


# In[17]:


class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        
        residual, batch_size = input_Q, input_Q.size(0)

        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]
        
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]
        
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        
        return nn.LayerNorm(d_model).to(device)(output + residual), attn


# In[18]:


class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]


    
    
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns



class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, dec_self_attn_mask): # dec_inputs = enc_outputs
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        '''
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn
    

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
#         self.tgt_emb = nn.Embedding(d_model * 2, d_model)
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])
        self.tgt_len = tgt_len
        
    def forward(self, dec_inputs): # dec_inputs = enc_outputs (batch_size, peptide_hla_maxlen_sum, d_model)
        '''
        dec_inputs: [batch_size, tgt_len]
        enc_intpus: [batch_size, src_len]
        enc_outputs: [batsh_size, src_len, d_model]
        '''
#         dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_inputs.transpose(0, 1)).transpose(0, 1).to(device) # [batch_size, tgt_len, d_model]
#         dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_pad_mask = torch.LongTensor(np.zeros((dec_inputs.shape[0], tgt_len, tgt_len))).bool().to(device)

        dec_self_attns = []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn = layer(dec_outputs, dec_self_attn_pad_mask)
            dec_self_attns.append(dec_self_attn)
            
        return dec_outputs, dec_self_attns


class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = dec_outputs.view(dec_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), pep_enc_self_attns, hla_enc_self_attns, dec_self_attns

In [71]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    try:
        mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    except:
        print('MCC Error: ', (tp+fn)*(tn+fp)*(tp+fp)*(tn+fn))
        mcc = np.nan
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = np.nan
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = np.nan
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = np.nan
        
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
        print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))
    
    return (roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr)



def performances_QM_(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('acc={:.4f}'.format(accuracy))
    
    return (fn, tp, accuracy)

def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])


f_mean = lambda l: sum(l)/len(l)


def performances_to_pd(performances_list):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']

    performances_pd = pd.DataFrame(performances_list, columns = metrics_name)
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    
    return performances_pd


In [72]:
def train_step(model, train_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    time_train_ep = 0
    model.train()
    y_true_train_list, y_prob_train_list = [], []
    loss_train_list, dec_attns_train_list = [], []
    for train_pep_inputs, train_hla_inputs, train_labels in train_loader:
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        train_outputs: [batch_size, 2]
        '''
        train_pep_inputs, train_hla_inputs, train_labels = train_pep_inputs.to(device), train_hla_inputs.to(device), train_labels.to(device)

        t1 = time.time()
        train_outputs, _, _, train_dec_self_attns = model(train_pep_inputs, train_hla_inputs)
        train_loss = criterion(train_outputs, train_labels)
        time_train_ep += time.time() - t1

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        y_true_train = train_labels.cpu().numpy()
        y_prob_train = nn.Softmax(dim = 1)(train_outputs)[:, 1].cpu().detach().numpy()
        
        y_true_train_list.extend(y_true_train)
        y_prob_train_list.extend(y_prob_train)
        loss_train_list.append(train_loss)
#         dec_attns_train_list.append(train_dec_self_attns)
        
    y_pred_train_list = transfer(y_prob_train_list, threshold)
    ys_train = (y_true_train_list, y_pred_train_list, y_prob_train_list)
    
    print('Fold-{}****Train (Ep avg): Epoch-{}/{} | Loss = {:.4f} | Time = {:.4f} sec'.format(fold, epoch, epochs, f_mean(loss_train_list), time_train_ep))
    metrics_train = performances(y_true_train_list, y_pred_train_list, y_prob_train_list, print_ = True)
    
    return ys_train, loss_train_list, metrics_train, time_train_ep#, dec_attns_train_list


def eval_step(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print('Fold-{} ****Test  Epoch-{}/{}: Loss = {:.6f}'.format(fold, epoch, epochs, f_mean(loss_val_list)))
        
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list

In [73]:
pep_max_len = 14 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

vocab = np.load( data_dir + 'Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)

# Transformer Parameters
d_model = 20  # Embedding Size
d_k = d_v = 20  # dimension of K(=Q), V

d_ff = 512 # FeedForward dimension
n_layers = 1  # number of Encoder of Decoder Layer
n_heads = 3

batch_size = 1024
epochs = 100
threshold = 0.5

In [74]:
###### -> upd
#index_order_nested

cv = 5
count = 0
index_order = []

for outer in range(cv):
        print(f"\nk_outer: {outer}")
        print("test: " + str(outer))
        test_idx = outer
        for inner in range(cv):
            print(f"\t\nk_inner: {inner}")
            if inner!=outer:
                print("\tval: " + str(inner))
                val_idx = inner
                train_index = list()
                for t in range(5):
                    if t!=inner and t!=outer:
                        train_index.append(t)
                print("\ttrain:", str(train_index))
                count += 1  
                index_order.append([test_idx, val_idx, train_index])
                
index_order_nested = index_order


###### -> upd - nested

def data_with_loader(index_order_idx, type_ = 'train', fold = None,  batch_size = 1024):
    df_list = []
    for i in range(5):
        df_list.append(pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0) #[0:per])
        
    if type_ != 'train' and type_ != 'val' and type_ != 'test':
        data = pd.read_csv(data_dir + '{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.concat([ df_list[index_order_nested[index_order_idx][2][0]], df_list[index_order_nested[index_order_idx][2][1]], df_list[index_order_nested[index_order_idx][2][2]] ]).reset_index().drop(["index"], axis=1)
        print("TRAIN ID:", index_order_nested[index_order_idx][2], len(data))
    elif type_ == 'test':
        data = df_list[index_order_nested[index_order_idx][0]]
        print("TEST ID:", index_order_nested[index_order_idx][0], len(data))
    elif type_ == 'val':
        data = df_list[index_order_nested[index_order_idx][1]]
        print("VAL ID:", index_order_nested[index_order_idx][1], len(data))
        
    pep_inputs, hla_inputs, labels = make_data(data)
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    print('Fold-{} Label info: Data {} = {} '.format(fold, type_, Counter(data.label)))
    
    del df_list, data
    gc.collect()
    return pep_inputs, hla_inputs, labels, loader

print("\n\n\n")
for i in index_order_nested:
    print(i)
    
index_order = 0


k_outer: 0
test: 0
	
k_inner: 0
	
k_inner: 1
	val: 1
	train: [2, 3, 4]
	
k_inner: 2
	val: 2
	train: [1, 3, 4]
	
k_inner: 3
	val: 3
	train: [1, 2, 4]
	
k_inner: 4
	val: 4
	train: [1, 2, 3]

k_outer: 1
test: 1
	
k_inner: 0
	val: 0
	train: [2, 3, 4]
	
k_inner: 1
	
k_inner: 2
	val: 2
	train: [0, 3, 4]
	
k_inner: 3
	val: 3
	train: [0, 2, 4]
	
k_inner: 4
	val: 4
	train: [0, 2, 3]

k_outer: 2
test: 2
	
k_inner: 0
	val: 0
	train: [1, 3, 4]
	
k_inner: 1
	val: 1
	train: [0, 3, 4]
	
k_inner: 2
	
k_inner: 3
	val: 3
	train: [0, 1, 4]
	
k_inner: 4
	val: 4
	train: [0, 1, 3]

k_outer: 3
test: 3
	
k_inner: 0
	val: 0
	train: [1, 2, 4]
	
k_inner: 1
	val: 1
	train: [0, 2, 4]
	
k_inner: 2
	val: 2
	train: [0, 1, 4]
	
k_inner: 3
	
k_inner: 4
	val: 4
	train: [0, 1, 2]

k_outer: 4
test: 4
	
k_inner: 0
	val: 0
	train: [1, 2, 3]
	
k_inner: 1
	val: 1
	train: [0, 2, 3]
	
k_inner: 2
	val: 2
	train: [0, 1, 3]
	
k_inner: 3
	val: 3
	train: [0, 1, 2]
	
k_inner: 4




[0, 1, [2, 3, 4]]
[0, 2, [1, 3, 4]]
[0, 3, [1, 2, 4

In [ ]:
ys_train_fold_dict, ys_val_fold_dict = {}, {}
train_fold_metrics_list, val_fold_metrics_list, test_fold_metrics_list = [], [], []
#independent_fold_metrics_list, external_fold_metrics_list, ys_independent_fold_dict, ys_external_fold_dict = [], [], {}, {}
attns_train_fold_dict, attns_val_fold_dict = {}, {}
loss_train_fold_dict, loss_val_fold_dict, = {}, {}
metrics_all_val = [] 
metrics_all_train = [] 

time_train_c = time.time()

for fold in tqdm(range(20)):  #####

        print('=====Inner Fold-{}====='.format(fold))
        print('-----Generate data loader-----')
        index_order_idx = fold
        train_pep_inputs, train_hla_inputs, train_labels, train_loader = data_with_loader(index_order_idx, type_ = 'train', fold = fold,  batch_size = batch_size)
        val_pep_inputs, val_hla_inputs, val_labels, val_loader = data_with_loader(index_order_idx, type_ = 'val', fold = fold,  batch_size = batch_size)

        print('TEST ID', index_order_nested[index_order_idx][0])

        print('-----Compile model-----')
        model = Transformer().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 1e-3) #, momentum = 0.99)

        print('-----Train-----')
        path_saver = model_folder + pkl(id_, n_layers, n_heads, fold, d_model)

        print('path_saver: ', path_saver)

        metric_best, ep_best = 0, -1
        time_train = 0

        #if not os.path.exists(path_saver):
        for epoch in range(1, epochs + 1):
            time_train_f = time.time()
            ys_train, loss_train_list, metrics_train, time_train_ep = train_step(model, train_loader, fold, epoch, epochs, use_cuda) # , dec_attns_train
            ys_val, loss_val_list, metrics_val, _prob, _binary = eval_step(model, val_loader, fold, epoch, epochs, use_cuda) #, dec_attns_val

            train_fold_metrics_list.append(metrics_train)
            val_fold_metrics_list.append(metrics_val)

            metrics_ep_avg = sum(metrics_val[:4])/4
            if metrics_ep_avg > metric_best: 
                metric_best, ep_best = metrics_ep_avg, epoch
                torch.save(model.eval().state_dict(), path_saver)
            print("Fold time: {:6.2f} mins.".format(   round( (time.time()-time_train_f)/60, 2)   ))

        print("best_epoch:", ep_best)
        print('-----Optimization Finished!-----')
print("Total training time: {:6.2f} mins.".format(   round( (time.time()-time_train_c)/60, 2)   ))


  0%|                                                                                                                                                         | 0/20 [00:00<?, ?it/s]

=====Inner Fold-0=====
-----Generate data loader-----
TRAIN ID: [2, 3, 4] 451500
Fold-0 Label info: Data train = Counter({1: 226074, 0: 225426}) 
VAL ID: 1 150500
Fold-0 Label info: Data val = Counter({0: 75567, 1: 74933}) 
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_25p_EmbeddingAttention_100ep_d20_layer1_multihead3_MODEL0.pkl
Fold-0****Train (Ep avg): Epoch-1/100 | Loss = 0.3161 | Time = 10.1062 sec
tn = 192308, fp = 33118, fn = 28784, tp = 197290
y_pred: 0 = 221092 | 1 = 230408
y_true: 0 = 225426 | 1 = 226074
auc=0.9391|sensitivity=0.8727|specificity=0.8531|acc=0.8629|mcc=0.7259
precision=0.8563|recall=0.8727|f1=0.8644|aupr=0.9368
Fold-0 ****Test  Epoch-1/100: Loss = 0.252641
tn = 69321, fp = 6246, fn = 8814, tp = 66119
y_pred: 0 = 78135 | 1 = 72365
y_true: 0 = 75567 | 1 = 74933
auc=0.9626|sensitivity=0.8824|specificity=0.9173|acc=0.8999|mcc=0.8003
precision=0.9137|recall=0.8824|f1=0.8978|aupr=0

Fold-0****Train (Ep avg): Epoch-14/100 | Loss = 0.1543 | Time = 11.9543 sec
tn = 211820, fp = 13606, fn = 13184, tp = 212890
y_pred: 0 = 225004 | 1 = 226496
y_true: 0 = 225426 | 1 = 226074
auc=0.9853|sensitivity=0.9417|specificity=0.9396|acc=0.9407|mcc=0.8813
precision=0.9399|recall=0.9417|f1=0.9408|aupr=0.9853
Fold-0 ****Test  Epoch-14/100: Loss = 0.228346
tn = 70139, fp = 5428, fn = 6953, tp = 67980
y_pred: 0 = 77092 | 1 = 73408
y_true: 0 = 75567 | 1 = 74933
auc=0.9730|sensitivity=0.9072|specificity=0.9282|acc=0.9177|mcc=0.8356
precision=0.9261|recall=0.9072|f1=0.9165|aupr=0.9723
Fold time:   0.85 mins.
Fold-0****Train (Ep avg): Epoch-15/100 | Loss = 0.1502 | Time = 13.6248 sec
tn = 212203, fp = 13223, fn = 12847, tp = 213227
y_pred: 0 = 225050 | 1 = 226450
y_true: 0 = 225426 | 1 = 226074
auc=0.9861|sensitivity=0.9432|specificity=0.9413|acc=0.9423|mcc=0.8845
precision=0.9416|recall=0.9432|f1=0.9424|aupr=0.9861
Fold-0 ****Test  Epoch-15/100: Loss = 0.229150
tn = 70212, fp = 5355, fn =

Fold-0 ****Test  Epoch-27/100: Loss = 0.287432
tn = 68629, fp = 6938, fn = 5515, tp = 69418
y_pred: 0 = 74144 | 1 = 76356
y_true: 0 = 75567 | 1 = 74933
auc=0.9725|sensitivity=0.9264|specificity=0.9082|acc=0.9173|mcc=0.8347
precision=0.9091|recall=0.9264|f1=0.9177|aupr=0.9718
Fold time:   0.80 mins.
Fold-0****Train (Ep avg): Epoch-28/100 | Loss = 0.1069 | Time = 12.5982 sec
tn = 216165, fp = 9261, fn = 9156, tp = 216918
y_pred: 0 = 225321 | 1 = 226179
y_true: 0 = 225426 | 1 = 226074
auc=0.9930|sensitivity=0.9595|specificity=0.9589|acc=0.9592|mcc=0.9184
precision=0.9591|recall=0.9595|f1=0.9593|aupr=0.9931
Fold-0 ****Test  Epoch-28/100: Loss = 0.293949
tn = 68720, fp = 6847, fn = 5691, tp = 69242
y_pred: 0 = 74411 | 1 = 76089
y_true: 0 = 75567 | 1 = 74933
auc=0.9723|sensitivity=0.9241|specificity=0.9094|acc=0.9167|mcc=0.8335
precision=0.9100|recall=0.9241|f1=0.9170|aupr=0.9715
Fold time:   0.85 mins.
Fold-0****Train (Ep avg): Epoch-29/100 | Loss = 0.1037 | Time = 11.5911 sec
tn = 216562, 

Fold-0****Train (Ep avg): Epoch-41/100 | Loss = 0.0803 | Time = 11.0471 sec
tn = 218222, fp = 7204, fn = 7005, tp = 219069
y_pred: 0 = 225227 | 1 = 226273
y_true: 0 = 225426 | 1 = 226074
auc=0.9961|sensitivity=0.9690|specificity=0.9680|acc=0.9685|mcc=0.9371
precision=0.9682|recall=0.9690|f1=0.9686|aupr=0.9962
Fold-0 ****Test  Epoch-41/100: Loss = 0.414944
tn = 67181, fp = 8386, fn = 5044, tp = 69889
y_pred: 0 = 72225 | 1 = 78275
y_true: 0 = 75567 | 1 = 74933
auc=0.9704|sensitivity=0.9327|specificity=0.8890|acc=0.9108|mcc=0.8224
precision=0.8929|recall=0.9327|f1=0.9123|aupr=0.9701
Fold time:   0.76 mins.
Fold-0****Train (Ep avg): Epoch-42/100 | Loss = 0.0797 | Time = 11.4315 sec
tn = 218216, fp = 7210, fn = 7027, tp = 219047
y_pred: 0 = 225243 | 1 = 226257
y_true: 0 = 225426 | 1 = 226074
auc=0.9961|sensitivity=0.9689|specificity=0.9680|acc=0.9685|mcc=0.9369
precision=0.9681|recall=0.9689|f1=0.9685|aupr=0.9962
Fold-0 ****Test  Epoch-42/100: Loss = 0.414609
tn = 67550, fp = 8017, fn = 518

...

Fold-5 ****Test  Epoch-67/100: Loss = 0.529751
tn = 69302, fp = 6103, fn = 6763, tp = 68332
y_pred: 0 = 76065 | 1 = 74435
y_true: 0 = 75405 | 1 = 75095
auc=0.9712|sensitivity=0.9099|specificity=0.9191|acc=0.9145|mcc=0.8291
precision=0.9180|recall=0.9099|f1=0.9140|aupr=0.9708
Fold time:   0.65 mins.
Fold-5****Train (Ep avg): Epoch-68/100 | Loss = 0.0516 | Time = 7.7069 sec
tn = 220418, fp = 4829, fn = 4766, tp = 221487
y_pred: 0 = 225184 | 1 = 226316
y_true: 0 = 225247 | 1 = 226253
auc=0.9984|sensitivity=0.9789|specificity=0.9786|acc=0.9787|mcc=0.9575
precision=0.9787|recall=0.9789|f1=0.9788|aupr=0.9984
Fold-5 ****Test  Epoch-68/100: Loss = 0.544718
tn = 69333, fp = 6072, fn = 6793, tp = 68302
y_pred: 0 = 76126 | 1 = 74374
y_true: 0 = 75405 | 1 = 75095
auc=0.9708|sensitivity=0.9095|specificity=0.9195|acc=0.9145|mcc=0.8291
precision=0.9184|recall=0.9095|f1=0.9139|aupr=0.9705
Fold time:   0.62 mins.
Fold-5****Train (Ep avg): Epoch-69/100 | Loss = 0.0514 | Time = 8.0051 sec
tn = 220509, fp

Fold-5****Train (Ep avg): Epoch-81/100 | Loss = 0.0416 | Time = 7.7665 sec
tn = 221428, fp = 3819, fn = 3792, tp = 222461
y_pred: 0 = 225220 | 1 = 226280
y_true: 0 = 225247 | 1 = 226253
auc=0.9989|sensitivity=0.9832|specificity=0.9830|acc=0.9831|mcc=0.9663
precision=0.9831|recall=0.9832|f1=0.9832|aupr=0.9990
Fold-5 ****Test  Epoch-81/100: Loss = 0.595364
tn = 68693, fp = 6712, fn = 6374, tp = 68721
y_pred: 0 = 75067 | 1 = 75433
y_true: 0 = 75405 | 1 = 75095
auc=0.9698|sensitivity=0.9151|specificity=0.9110|acc=0.9130|mcc=0.8261
precision=0.9110|recall=0.9151|f1=0.9131|aupr=0.9697
Fold time:   0.65 mins.
Fold-5****Train (Ep avg): Epoch-82/100 | Loss = 0.0413 | Time = 7.7278 sec
tn = 221523, fp = 3724, fn = 3764, tp = 222489
y_pred: 0 = 225287 | 1 = 226213
y_true: 0 = 225247 | 1 = 226253
auc=0.9990|sensitivity=0.9834|specificity=0.9835|acc=0.9834|mcc=0.9668
precision=0.9835|recall=0.9834|f1=0.9835|aupr=0.9990
Fold-5 ****Test  Epoch-82/100: Loss = 0.594043
tn = 68827, fp = 6578, fn = 6464,

Fold-5 ****Test  Epoch-94/100: Loss = 0.643797
tn = 69503, fp = 5902, fn = 6968, tp = 68127
y_pred: 0 = 76471 | 1 = 74029
y_true: 0 = 75405 | 1 = 75095
auc=0.9702|sensitivity=0.9072|specificity=0.9217|acc=0.9145|mcc=0.8290
precision=0.9203|recall=0.9072|f1=0.9137|aupr=0.9701
Fold time:   0.64 mins.
Fold-5****Train (Ep avg): Epoch-95/100 | Loss = 0.0350 | Time = 7.5139 sec
tn = 222113, fp = 3134, fn = 3095, tp = 223158
y_pred: 0 = 225208 | 1 = 226292
y_true: 0 = 225247 | 1 = 226253
auc=0.9992|sensitivity=0.9863|specificity=0.9861|acc=0.9862|mcc=0.9724
precision=0.9862|recall=0.9863|f1=0.9862|aupr=0.9993
Fold-5 ****Test  Epoch-95/100: Loss = 0.662083
tn = 69842, fp = 5563, fn = 7383, tp = 67712
y_pred: 0 = 77225 | 1 = 73275
y_true: 0 = 75405 | 1 = 75095
auc=0.9703|sensitivity=0.9017|specificity=0.9262|acc=0.9140|mcc=0.8282
precision=0.9241|recall=0.9017|f1=0.9127|aupr=0.9702
Fold time:   0.63 mins.
Fold-5****Train (Ep avg): Epoch-96/100 | Loss = 0.0336 | Time = 7.4293 sec
tn = 222257, fp

 30%|█████████████████████████████████████████▍                                                                                                | 6/20 [7:56:06<17:21:33, 4463.85s/it]

y_pred: 0 = 74417 | 1 = 76083
y_true: 0 = 75405 | 1 = 75095
auc=0.9704|sensitivity=0.9215|specificity=0.9087|acc=0.9151|mcc=0.8302
precision=0.9095|recall=0.9215|f1=0.9155|aupr=0.9706
Fold time:   0.65 mins.
best_epoch: 13
-----Optimization Finished!-----
=====Inner Fold-6=====
-----Generate data loader-----
TRAIN ID: [0, 2, 4] 451500
Fold-6 Label info: Data train = Counter({1: 226151, 0: 225349}) 
VAL ID: 3 150500
Fold-6 Label info: Data val = Counter({0: 75303, 1: 75197}) 
TEST ID 1
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_25p_EmbeddingAttention_100ep_d20_layer1_multihead3_MODEL6.pkl
Fold-6****Train (Ep avg): Epoch-1/100 | Loss = 0.3190 | Time = 7.7616 sec
tn = 192157, fp = 33192, fn = 29421, tp = 196730
y_pred: 0 = 221578 | 1 = 229922
y_true: 0 = 225349 | 1 = 226151
auc=0.9380|sensitivity=0.8699|specificity=0.8527|acc=0.8613|mcc=0.7227
precision=0.8556|recall=0.8699|f1=0.8627|aupr=0.9360
Fold-6 ****Tes

In [ ]:
def data_with_loader_unique(index_order_idx, type_ = 'train', fold = None,  batch_size = 1024):
    df_list = []
    for i in range(5):
        df = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0).drop_duplicates()
        df_list.append(df)
        
    if type_ != 'train' and type_ != 'val' and type_ != 'test':
        data = pd.read_csv(data_dir + '{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.concat([ df_list[index_order_nested[index_order_idx][2][0]], df_list[index_order_nested[index_order_idx][2][1]], df_list[index_order_nested[index_order_idx][2][2]] ]).reset_index().drop(["index"], axis=1)
        print("TRAIN ID:", index_order_nested[index_order_idx][2], len(data))
    elif type_ == 'test':
        data = df_list[fold]
        print("TEST ID:", fold, len(data))
    elif type_ == 'val':
        data = df_list[index_order_nested[index_order_idx][1]]
        print("VAL ID:", index_order_nested[index_order_idx][1], len(data))
        
    pep_inputs, hla_inputs, labels = make_data(data)
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    print('Fold-{} Label info: Data {} = {} '.format(fold, type_, Counter(data.label)))
    
    del df_list
    gc.collect()
    return data, pep_inputs, hla_inputs, labels, loader

def eval_step_test(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)
            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print_bool = False
        if epoch == 1 or epoch == 25 or epoch == 50:
            print_bool = True
            print('\nFold-{} **** Test  Epoch-{}/{}: Loss = {:.6f} | '.format(fold, epoch, epochs, f_mean(loss_val_list)), end='')
            
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list, print_ = print_bool)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list




In [ ]:
id_nested = id_

ep_best = 0
criterion = nn.CrossEntropyLoss()

test_fold_metrics_list_nested = []

fold_avg_4 = True
type_ = 'test'

df_list = []
time_testing = time.time()

if fold_avg_4:
    
    fold_avg = [list(range(0,4)), list(range(4,8)), list(range(8,12)), list(range(12,16)), list(range(16,20))] 
    pred_all = []
    target_all = []

    for fold in fold_avg:

            pred_fold = []
            target_fold = []
            file_num = fold[0]
            print('\n=====Models-{}====='.format(fold))
            
            for f in fold:
                index_order_idx = f
                test_data, test_pep_inputs, test_hla_inputs, test_labels, test_loader = data_with_loader_unique(index_order_idx, type_ = type_, fold = file_num,  batch_size = batch_size)
                model = Transformer().to(device)
                path_saver = model_folder + pkl(id_nested, n_layers, n_heads, f, d_model)
                print(path_saver)
                model.load_state_dict(torch.load(path_saver))
                model_test = model.eval()

                ys_res_test, loss_res_test_list, metrics_res_test, y_prob, y_pred_bin = eval_step_test(model_test, test_loader, fold, ep_best, epochs, use_cuda) # , test_res_attns
                pred_fold.append(y_prob)
                target_fold = [x.item() for x in test_labels]
                
                ## add auc01 and ppv
                df_temp = pd.DataFrame(list(zip(target_fold, y_prob)), columns= ['target_', 'pred_'])
                num_pos = len(df_temp[df_temp['target_']==1])
                df_temp_ppv = df_temp.sort_values(by=['pred_'], ascending=False)[0:num_pos]
                num_true_pos = len(df_temp_ppv[df_temp_ppv['target_']==1])
                ppv = num_true_pos/num_pos
                auc_01 = binary_roc_auc_score(df_temp['target_'].tolist(), df_temp['pred_'], max_fpr=0.1)
                auc01_ppv = (auc_01, ppv)
                metrics_final = (*auc01_ppv, *metrics_res_test)
                
                test_fold_metrics_list_nested.append(metrics_final)
                
                print("\t\t\t\t roc in:", roc_auc_score(target_fold, y_prob))

            # get avg of pred_fold and extend pred_all
            arrays = [np.array(x) for x in pred_fold]
            pred_fold_avg = [np.mean(k) for k in zip(*arrays)]
            pred_all.extend(pred_fold_avg)
            target_all.extend(target_fold)
            print("\t\t roc avg accmmltd:", roc_auc_score(target_all, pred_all))  
            
            test_data['target_'] = target_fold
            test_data['pred_'] = pred_fold_avg
            df_list.append(test_data)


    df_pd_nested_test_unique = pd.concat(df_list)
    del df_list
    gc.collect()
    
print("Time testing:", round((time.time()-time_testing)/60, 2), "mins. \n\n")

In [ ]:
df_results = pd.DataFrame(list(zip(target_all, pred_all)), columns= ['target', 'pred'])
print("\nSTATS:")
auc_ = roc_auc_score(df_results['target'], df_results['pred'])
print("AUC:", auc_)

num_pos = len(df_results[df_results['target']==1])
df_temp_ppv = df_results.sort_values(by=['pred'], ascending=False)[0:num_pos]
num_true_pos = len(df_temp_ppv[df_temp_ppv['target']==1])
ppv = num_true_pos/num_pos
auc_01 = binary_roc_auc_score(df_results['target'].tolist(), df_results['pred'].tolist(), max_fpr=0.1)
print("AUC FPR 0.1:", auc_01)
print("PPV:", ppv)

print("\nPerformance adv:")

acc_best, i, df_results = best_treshold(df_results, [0.5])
print("Used/Best treshold:", i, "\n")
print("AUC pred treshold:", roc_auc_score(df_results['target'], df_results['pred_binary']))
roc_auc, acc, mcc, f1, sensitivity, specificity, precision, recall, aupr = performances(df_results['target'], df_results['pred_binary'], df_results['pred'], print_ = True)

#df_pd_nested_test_unique.to_csv(df_name + 'test_avg4.csv', index=False)


In [18]:
file = open(df_name + 'test_avg4.csv', "w")

file.write(f"#INFO:\n")
file.write(f"#Test type: Every for of 20 model is tested on a partition, avarage is taken.\n")
file.write(f"#id: {id_}\n")
file.write(f"#d_model: {d_model}\n")
file.write(f"#d_k, d_v: {d_k}, {d_v}\n")
file.write(f"#n_heads: {n_heads}\n")
file.write(f"#n_layers: {n_layers}\n")
file.write(f"#ff_layers: {d_ff}\n")
file.write(f"#epochs: {epochs}\n")
file.write(f"#treshold: {threshold}\n")

file.write(''.join("peptide,length,HLA,label,HLA_sequence,target_,pred_\n"))

for index, row in df_pd_nested_test_unique.iterrows():
    txt = row['peptide'] + "," + str(row['length']) + "," + row['HLA'] + "," + str(row['label']) + "," + row['HLA_sequence'] + "," + str(row['target_']) + "," + str(row['pred_']) + "\n"
    file.write(''.join(txt))

file.close()

print("DONE")

DONE


In [17]:
#save in global file:
attr = ['id', 'data', 'data_size', 'data_info', 'encoding', 'results_df', 'pep_max_len', 'hla_max_len', 'tgt_len', 'd_model', 'd_k', 'd_v', 'd_ff', 'n_layers', 'n_heads', 'batch_size', 'epochs', 'treshold', 'test_dup', 'test_type', 'auc', 'auc01', 'ppv', 'acc', 'mcc', 'sensitivity', 'specificity', 'precision', 'recall', 'f1', 'aupr']

test_dup = 'tested on unique'

global_info = [id_, data_ , size, per, encoding_ , df_name + 'test_avg4.csv', pep_max_len, hla_max_len, tgt_len, d_model, d_k, d_v, d_ff, n_layers, n_heads, batch_size, epochs, threshold, test_dup, 'average of each 4 - then concat', auc_, auc_01, ppv, acc, mcc, sensitivity, specificity, precision, recall, f1, aupr]

for i,t in zip(global_info, attr):
    print(t, ":", i)
    
with open(result_folder + 'results_global', 'a') as file:
    file.write('\n'+','.join([str(x) for x in global_info]))
file.close()

id : OF_TData_pad14_75p_EmbeddingAttention
data : TransMut
data_size : 75p
data_info : 451500
encoding : embedding
results_df : /home/s202357/thesis/transmut/pipeline/original/results/OF_TData_pad14_75p_EmbeddingAttention_df_test_avg4.csv
pep_max_len : 14
hla_max_len : 34
tgt_len : 48
d_model : 20
d_k : 20
d_v : 20
d_ff : 512
n_layers : 1
n_heads : 3
batch_size : 1024
epochs : 25
treshold : 0.5
test_dup : tested on unique
test_type : average of each 4 - then concat
auc : 0.9800021812742524
auc01 : 0.8600724906104735
ppv : 0.8834800621439669
acc : 0.9306493868319099
mcc : 0.8235768368951786
sensitivity : 0.9311349069789456
specificity : 0.9304946286180839
precision : 0.8102512168353317
recall : 0.9311349069789456
f1 : 0.8664973047620405
aupr : 0.9468830296400268
